# This Notebook is for transformer study from scratch

In [2]:
import torch
import numpy as np

In [25]:
with open('input.txt', 'r', encoding = 'utf-8') as f:
    text = f.read()

In [26]:
print(text[:1000]), print(f'No of Characters we are dealing with: {len(text)}')

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



(None, None)

In [5]:
len(text)

1115394

In [6]:
chars = sorted(list(set(text)))
''.join(chars)

"\n !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz"

In [96]:
len(chars)

65

In [7]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}

In [8]:
encode = lambda s : [stoi[c] for c in s]
decode = lambda i : ''.join([itos[n] for n in i])

In [9]:
print(encode('Hello'))
print(decode(encode('Hello')))

[20, 43, 50, 50, 53]
Hello


# Here we will encode the shakesperian text in encoded form to be processed by the model

In [27]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([1115394]) torch.int64
tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56,  1, 51, 43,  1,
        57, 54, 43, 39, 49,  8,  0,  0, 13, 50, 50, 10,  0, 31, 54, 43, 39, 49,
         6,  1, 57, 54, 43, 39, 49,  8,  0,  0, 18, 47, 56, 57, 58,  1, 15, 47,
        58, 47, 64, 43, 52, 10,  0, 37, 53, 59,  1, 39, 56, 43,  1, 39, 50, 50,
         1, 56, 43, 57, 53, 50, 60, 43, 42,  1, 56, 39, 58, 46, 43, 56,  1, 58,
        53,  1, 42, 47, 43,  1, 58, 46, 39, 52,  1, 58, 53,  1, 44, 39, 51, 47,
        57, 46, 12,  0,  0, 13, 50, 50, 10,  0, 30, 43, 57, 53, 50, 60, 43, 42,
         8,  1, 56, 43, 57, 53, 50, 60, 43, 42,  8,  0,  0, 18, 47, 56, 57, 58,
         1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 18, 47, 56, 57, 58,  6,  1, 63,
        53, 59,  1, 49, 52, 53, 61,  1, 15, 39, 47, 59, 57,  1, 25, 39, 56, 41,
      

# Here we have spilitted the data into training and validation set

In [28]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [12]:
# For Understanding block_size
# block_size = 8
# train_data = data[: block_size+1]
# print(train_data)
# x = train_data[:block_size]
# y = train_data[1:block_size+1]
# for i in range(block_size):
#     context = x[:i+1]
#     print(f'You trained on {context}: {y[i]}')

tensor([18, 47, 56, 57, 58,  1, 15, 47, 58])
You trained on tensor([18]): 47
You trained on tensor([18, 47]): 56
You trained on tensor([18, 47, 56]): 57
You trained on tensor([18, 47, 56, 57]): 58
You trained on tensor([18, 47, 56, 57, 58]): 1
You trained on tensor([18, 47, 56, 57, 58,  1]): 15
You trained on tensor([18, 47, 56, 57, 58,  1, 15]): 47
You trained on tensor([18, 47, 56, 57, 58,  1, 15, 47]): 58


Here we will split the data into batches for parallel processing in GPU

In [14]:
print(x)
print(y)

tensor([18, 47, 56, 57, 58,  1, 15, 47])
tensor([47, 56, 57, 58,  1, 15, 47, 58])


# Dividing the data into batches

In [32]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8
def get_batch(split = 'train'):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

In [91]:
xb, yb = get_batch('train')
print(xb)
print(yb)

tensor([[53,  1, 61, 39, 47, 58,  1, 59],
        [ 1, 58, 47, 51, 43, 57,  1, 47],
        [63,  1, 50, 53, 56, 42, 11,  1],
        [41, 47, 53, 59, 57,  1, 50, 47]])
tensor([[ 1, 61, 39, 47, 58,  1, 59, 54],
        [58, 47, 51, 43, 57,  1, 47, 52],
        [ 1, 50, 53, 56, 42, 11,  1, 21],
        [47, 53, 59, 57,  1, 50, 47, 60]])


In [101]:
# Here we will define a bigrams and get the word embeddings
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super(BigramLanguageModel, self).__init__() 
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets= None): # This function just operate like __call__
        if target is None:
            loss = None
        logits = self.token_embedding_table(idx) # (Batch, Time, Channels) (4, 8(block_size), 65(vocab_size))

        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens): # With the help of this function we will predict the next character
        for _ in range(max_new_tokens):
            logits, loss = self(idx)
            logits = logits[:,-1,:] # extracting the last character from the embedding this wll of the shape (B, 65). so to predict its next element
            prob = F.softmax(logits, dim = -1)
            idx_next = torch.multinomial(prob, num_samples = 1) #(B,1)
            idx = torch.cat((idx, idx_next), dim = 1) # (B, T+1)
        return idx
        

m = BigramLanguageModel(len(chars))
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

torch.Size([32, 65])
tensor(4.7707, grad_fn=<NllLossBackward0>)
